In [20]:
from typing import Optional
from dataclasses import dataclass, field

class Student(Library):
    """Creates the actions of the students"""

    def __init__(self, name: str, year: str) -> None:
        self.name = name
        self.year = year
        self.balance = 0
        self.loaned_books = {}

        super().__init__()

    def __str__(self) -> str:
        """Displays the actual information about the student"""
        return f"Name: {self.name}\nClass: {self.year}\nBalance: ${self.balance}\nLoaned books: {self.loaned_books}"


class Library:
    """Creates the operations of the library"""

    def __init__(self) -> None:
        
        self.collection = {}
        self.registrated = {}

In [23]:
bob = Stud("Tom Jerry", 2023)

In [24]:
bob.__dict__

{'name': 'Tom Jerry', 'grade': 2023, 'account': 0, 'loaned_books': {}}

In [67]:
from datetime import datetime
from dataclasses import dataclass, field

@dataclass
class Library:
    """Creates the operations of the library"""

    #def __init__(self) -> None:

    collection: dict = field(default_factory=dict)
    registrated: dict = field(default_factory=dict)

    def registrate(self, student: Student):
        """Student registration"""

        # Creates the dictionary about the student data for the library
        # Dict -> Name: Class, Balance, Loan informations
        student = student.__dict__

        self.registrated[student["name"]] = {
            "class": student["year"],
            "balance": student["balance"],
            "loaned_books": student["loaned_books"],
        }

        print(
            "{} from class of {} is registrated".format(
                student["name"], student["year"]
            )
        )
        print("Library balance: ${}".format(student["balance"]))

    def add_book(self, book: str, info: dict) -> None:
        """Add books to the library"""

        self.collection[book] = info

    def remove_book(self, book: str) -> None:
        """Remove books from the library"""

        del self.collection[book]

@dataclass
class Student:
    """Creates the actions of the students"""

    name: str
    year: int 
    info: Library
    loaned_books: dict = field(default_factory=dict)
    balance: int = 0

    def __str__(self) -> str:
        """Displays the actual information about the student"""
        
        return f"Name: {self.name}\nClass: {self.year}\nBalance: ${self.balance}\nLoaned books: {self.loaned_books}"

    def deposit_money(self, money):
        """Student deposit on the account"""

        self.balance += money

        return f"Your balance: ${self.balance}"

    def loan_book(self, books: list, date: str):
        """Loaning process"""
        """Argument "books" type is list if the loan request contains several books """

        if self.name not in self.info.registrated or self.balance < 0:

            return f"{self.name} is not registrated in the system yet or your balance is negative. Please contact with the reception"

        elif len(books) > 10:

            return f"Maximum number of loan request is 10!\n Your request number: {len(books)}"

        for book in books:

            loaned = []  # collect the available book
            if book in self.info.collection:

                loaned.append(book)  # add the available book to the list

                # register the loan details(book and date) the student side
                self.loaned_books[book] = self.info.collection[book]
                self.loaned_books[book]["loan_date"] = datetime.strptime(
                    date, "%Y-%m-%w"
                )
                # register the loan details(book and date) the library side
                self.info.registrated[self.name]["loaned_books"][book] = self.info.collection[book]
                self.info.registrated[self.name]["loaned_books"][book][
                    "loan_date"
                ] = datetime.strptime(date, "%Y-%m-%w")

                # remove the book temporarely from the library
                self.info.remove_book(book)

            print(f"The book {book} is not available!")

        return f"Successfully loaned:{loaned}"

    def return_book(self, books: list, date: str):  # argument "books"
        """Book returning process"""
        """Argument "books" type is list if the returning process contains several books """

        for book in books:

            if book in self.info.registrated[self.name]["loaned_books"]:

                loan_date = self.info.registrated[self.name]["loaned_books"][
                    "loan_date"
                ]  # start date of the loan
                return_date = datetime.strptime(date, "%Y-%m-%w")  # return date

                diff = return_date - loan_date  # difference start date and return date

                # book get back to the library
                self.info.add_book(book, self.info.registrated[self.name]["loaned_books"][book])

                # loaned book and its loan date is getting deleted from the system
                del self.info.registrated[self.name]["loaned_books"][book]
                del self.info.registrated[self.name]["loaned_books"][book]["loan_date"]

                del self.loaned_books[book]
                del self.loaned_books[book]["loan_date"]

                # if the loaning time exceeds the 30 day rule
                if diff > 30:

                    self.balance -= diff

            print(f"The book {book} has not been loaned yet!")

        print(f"Return is successfully processed!")

In [68]:
library = Library()
tom = Student("Tom Jerry", 2023, library)

library.registrate(tom)

Tom Jerry from class of 2023 is registrated
Library balance: $0


In [69]:
library.registrated

{'Tom Jerry': {'class': 2023, 'balance': 0, 'loaned_books': {}}}

In [70]:
library.add_book("Harry Potter", {"date":2001,"genre":"fantasy"})

In [71]:
library.collection

{'Harry Potter': {'date': 2001, 'genre': 'fantasy'}}

In [72]:
tom.loan_book(["Harry Potter"], "2023-01-24")

ValueError: unconverted data remains: 4

In [73]:
'Tom Jerry' in library.registrated

True

In [74]:
tom.info.registrated

{'Tom Jerry': {'class': 2023,
  'balance': 0,
  'loaned_books': {'Harry Potter': {'date': 2001, 'genre': 'fantasy'}}}}